In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import csv
import string
from imblearn.over_sampling import SMOTE
from gensim.models import Word2Vec
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, Concatenate
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/datasets/sgp1.csv'


df = pd.read_csv(dataset_path, delimiter=',', quoting=3, error_bad_lines=False)


relevant_columns = ['jobdescription', 'jobtitle', 'skills1']


df['combined_text'] = df[relevant_columns].apply(lambda x: ' '.join(map(str, x)), axis=1)


text_data = df['combined_text']


sentences = text_data.tolist()


tokenizer = Tokenizer()


tokenizer.fit_on_texts(sentences)


total_words = len(tokenizer.word_index) + 1


sequences = tokenizer.texts_to_sequences(sentences)

Streaming output truncated to the last 5000 lines.
Skipping line 1200: expected 40 fields, saw 49
Skipping line 1201: expected 40 fields, saw 47
Skipping line 1202: expected 40 fields, saw 41
Skipping line 1205: expected 40 fields, saw 47
Skipping line 1206: expected 40 fields, saw 46
Skipping line 1207: expected 40 fields, saw 43
Skipping line 1209: expected 40 fields, saw 68
Skipping line 1210: expected 40 fields, saw 53
Skipping line 1211: expected 40 fields, saw 66
Skipping line 1212: expected 40 fields, saw 43
Skipping line 1214: expected 40 fields, saw 47
Skipping line 1218: expected 40 fields, saw 47
Skipping line 1219: expected 40 fields, saw 41
Skipping line 1223: expected 40 fields, saw 44
Skipping line 1226: expected 40 fields, saw 67
Skipping line 1229: expected 40 fields, saw 62
Skipping line 1230: expected 40 fields, saw 44
Skipping line 1231: expected 40 fields, saw 48
Skipping line 1232: expected 40 fields, saw 43
Skipping line 1235: expected 40 fields, saw 60
Skipping 

In [ ]:

selected_columns = [ 'jobdescription', 'skills1', 'jobtitle']


df[selected_columns].fillna("", inplace=True)


label_encoders = {}
for col in selected_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

X = df[selected_columns]
y = df['companyname']

from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['companyname'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


input_layers = []
embedding_layers = []

for col in selected_columns:
    num_unique = len(df[col].unique())
    input_layer = Input(shape=(1,))
    embedding_layer = Embedding(num_unique, 10)(input_layer)  # You can adjust the embedding size as needed.
    input_layers.append(input_layer)
    embedding_layers.append(embedding_layer)


concatenated = Concatenate()(embedding_layers)
flatten = Flatten()(concatenated)
dense_layer = Dense(128, activation='relu')(flatten)
output_layer = Dense(len(df['companyname'].unique()), activation='softmax')(dense_layer)

<ipython-input-7-f9cfd14d77be>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_columns].fillna("", inplace=True)


In [ ]:
y_test

array([892, 994, 994, ..., 994, 994, 994])

In [ ]:
model = Model(inputs=input_layers, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(df.dtypes)

unique_labels = df['companyname'].nunique()
print("Number of unique labels in 'company' column:", unique_labels)


model.fit([X_train[col] for col in selected_columns], y_train, epochs=5, batch_size=64, validation_split=0.3)


loss, accuracy = model.evaluate([X_test[col] for col in selected_columns], y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")

import numpy as np
from sklearn.preprocessing import LabelEncoder




companyname       object
jobdescription     int64
jobtitle           int64
skills1            int64
combined_text     object
dtype: object
Number of unique labels in 'company' column: 994
Epoch 1/5
146/146 [==============================] - 3s 11ms/step - loss: 3.3893 - accuracy: 0.9013 - val_loss: 0.6030 - val_accuracy: 0.9298
Epoch 2/5
146/146 [==============================] - 1s 8ms/step - loss: 0.6463 - accuracy: 0.9204 - val_loss: 0.6168 - val_accuracy: 0.9298
Epoch 3/5
146/146 [==============================] - 1s 10ms/step - loss: 0.5934 - accuracy: 0.9209 - val_loss: 0.6582 - val_accuracy: 0.9298
Epoch 4/5
146/146 [==============================] - 2s 12ms/step - loss: 0.5560 - accuracy: 0.9212 - val_loss: 0.6927 - val_accuracy: 0.9298
Epoch 5/5
105/105 [==============================] - 0s 3ms/step - loss: 0.7757 - accuracy: 0.9259
Test Loss: 0.7757, Test Accuracy: 92.59%


In [ ]:
# Assuming label_encoder is your LabelEncoder instance
label_encoder = LabelEncoder()
# Fit and transform your labels
encoded_labels = label_encoder.fit_transform(df['companyname'])

# Save the label encoder to a file
np.save('label_encoder_company.npy', label_encoder.classes_)

In [ ]:
model.save('/content/drive/MyDrive/datasets/ANN2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Embedding, Dense, Input, Flatten, Concatenate


loaded_model = load_model('/content/drive/MyDrive/datasets/ANN_92.h5')


label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder_company.npy', allow_pickle=True)


new_data = pd.DataFrame(columns=['jobdescription', 'jobtitle', 'skills'])


new_data.loc[0] = ['java',' python',' c++']


new_data['combined_text'] = new_data[['jobdescription', 'jobtitle', 'skills']].apply(lambda x: ' '.join(map(str, x)), axis=1)


tokenizer = Tokenizer()


tokenizer.fit_on_texts(sentences)


new_sequences = tokenizer.texts_to_sequences(new_data['combined_text'])


max_sequence_length = len(max(sequences, key=len))
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)


selected_columns = ['jobdescription', 'skills', 'jobtitle']

print([new_padded_sequences[:, tokenizer.word_index.get(col, 2)] for col in selected_columns])


predictions = loaded_model.predict([new_padded_sequences[:, tokenizer.word_index.get(col, 2)] for col in selected_columns])


predicted_labels_encoded = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded)


print(predicted_labels)


[array([0], dtype=int32), array([0], dtype=int32), array([0], dtype=int32)]
1/1 [==============================] - 0s 290ms/step
[' minorities']
